# Visualization Pipeline Version 0.2.3

### Instructions

- Enter inputs, press `Save Inputs` button, check if the inputs were validated. If they were not validated see output explanation. Change input and press `Save Inputs` again. If any variables do not apply to your application, they should be completely empty.
- Press `Run Pipeline`, the progress indicator bar at the bottom indicates how many data subsets have been processed.
- The UI will display links to view the images of each data subset in AVIATOR.
- **Important:** The input and output directories must both be in one of the following `/home/jovyan/shared/data/` (recommended) or `/home/jovyan/shared/notebooks/` or `/home/jovyan/work/`

### Input Descriptions
- `inpDir`: Full path to the input images, should start with `/home/jovyan/`
- `outDir`: Full path to output, if you want to view images in AVIATOR then it should be in `/home/jovyan/shared/data/file-server/`
- `logPath`: Full path to this dashboard log outputs
- `filePattern`: The file pattern of the input images (Include ALL variables)
- `subsetBy`: The variable used to subset the data. For instance, you might process each plate individually to have a final Zarr pyramid for each plate.
- `depthVar`: The variable which represents a third dimension as opposed to width and height. This could be channel, time slice etc.
- `layout`: How to images should be assembled. This is the montage plugin layout. Please see https://github.com/PolusAI/polus-plugins/tree/master/transforms/images/polus-montage-plugin for more details. Note neither the `depthVar` or `subsetBy` variables can be included in the layout.
- `flipAxis`: The vaiable if any which should be flipped when assembling the images. For instance, if the x varible represents the width dimension and the images are flipped from right to left then the input would simple be `x`
- `subsets`: Which data subsets to process, for example if you want to process the first 3 plates you can input `1,2,3` or `all` to process all subsets

### Version Notes
- This version supports parallel execution. Each worker is responsible for starting the workflow (for a given subset) and monitoring its containers so that it can start the next container in the workflow upon completion of the previous container. 
- This version allows for the center FOV to be used for basic flatfield estimation or all images.
- Ability to resume a job by parsing the output directory, there is also functionality to parse the logs in this file but it not currently used.
- Ability to select if flatfield correction should be run or not.


In [1]:
import os
import sys
import shutil
import re
import subprocess
import ipywidgets as widgets
import multiprocessing as mp
from time import sleep
from pathlib import Path
from IPython.display import display
from filepattern import FilePattern
from multiprocessing import Pool
from datetime import datetime


def parallelize_workflow(map_args, ui, log_path):
    hashes = []
    num_workers = min(int(mp.cpu_count()//2), 4)
    with ui.out:
        print('Number of workers = {}'.format(num_workers))
        print('Spawning workers...')
        complete = 0

        with Pool(processes=num_workers) as pool:
            for status, subset, subset_hashes in pool.imap(run_workflow, map_args):
                hashes.extend(subset_hashes)
                with open(log_path, 'a') as fw:
                    if status:
                        fw.write('{} : subset {} successful : hashes : {}\n'.format(datetime.now(), subset, ' '.join(subset_hashes)))

                        # Stop the successful containers upon completion
                        stopped_containers = stop_containers(subset_hashes)
                        print('Stopped containers of subset {} : hashes : {}'.format(subset, ' '.join(stopped_containers)))

                    else:
                        fw.write('ERROR : {} : subset {} failed : hashes : {}\n'.format(datetime.now(), subset, ' '.join(subset_hashes)))

                    fw.close()

                complete += 1
                ui.progress_bar.value = complete

    return hashes


def check_container(container_hash):
    """
    Checks the status of a docker container.
    """
    cmd = 'docker ps'
    out = subprocess.check_output(cmd, shell=True).decode(sys.stdout.encoding).split('\n')
    containers = [c.split() for c in out]
    status = 'Unknown'
    # with self.out:
    #     print(containers)
    for c in containers:
        if container_hash in c:
            status = c[-2]
            break
    return status


def run_container(args, volume):
    """
    Runs a docker container.
    """
    dock8r = 'docker run -v'

    jvolume, dvolume = volume.split(':')

    container = args['container']
    plugin_args = [
        '--' + arg + ' ' + str(value)
        for arg, value in args.items() if arg != 'container' and value != ''
    ]
    for i, arg in enumerate(plugin_args):
        if jvolume in arg:
            plugin_args[i] = arg.replace(jvolume, dvolume)

    cmd_list = [dock8r] + [volume] + [container] + plugin_args
    cmd = ' '.join(cmd_list)

    full_hash = subprocess.check_output(cmd, shell=True).decode(sys.stdout.encoding).strip()
    container_hash = full_hash.split('-')[-1]
    print('Running {} hash:{}'.format(container, container_hash))

    status = 'Unknown'
    i = 0

    while status != 'Succeeded':
        sleep(5)
        status = check_container(container_hash)
        print('Status of Container {} is {} : {}'.format(container_hash, status, datetime.now()))
        i += 1
        if i > 20 and status == 'Pending':
            return 'Pending', container_hash

        if status == 'Failed':
            return 'Failed', container_hash

    return 'Succeeded', container_hash


def run_workflow(plugin_args):
    """
    Controls the visualization workflow for a data subset.
    """

    hashes = []
    volume = plugin_args['volume']
    subset = plugin_args['subset']
    run_flatfield = plugin_args['run_flatfield']

    try:

        if run_flatfield:

            # Run the basic flatfiled for specified subset
            print('Running Basic Flatfield Plugin on subset {}'.format(subset))
            basic_flatfield_args = plugin_args['BasicFlatfieldCorrectionPlugin']
            basic_flatfield_args['filePattern'] = basic_flatfield_args['filePattern'][subset]
            basic_flatfield_args['outDir'] = basic_flatfield_args['outDir'][subset]
            # pprint(basic_flatfield_args)
            status, container_hash = run_container(basic_flatfield_args, volume)
            hashes.append(container_hash)
            if status != 'Succeeded':
                raise Exception('Status of container {} is {}'.format(
                    container_hash, status)
                               )

            # Run the apply flatfield plugin for specified subset
            print('Running Apply Flatfield Plugin on subset {}'.format(subset))
            apply_flatfield_args = plugin_args['ApplyFlatfield']
            apply_flatfield_args['imgPattern'] = apply_flatfield_args['imgPattern'][subset]
            apply_flatfield_args['ffDir'] = basic_flatfield_args['outDir']
            apply_flatfield_args['brightPattern'] = apply_flatfield_args['brightPattern'][subset]
            apply_flatfield_args['darkPattern'] = apply_flatfield_args['darkPattern'][subset]
            apply_flatfield_args['outDir'] = apply_flatfield_args['outDir'][subset]
            # pprint(apply_flatfield_args)
            status, container_hash = run_container(apply_flatfield_args, volume)
            hashes.append(container_hash)
            if status != 'Succeeded':
                raise Exception('Status of container {} is {}'.format(
                    container_hash, status)
                               )

        else:
            basic_flatfield_args = plugin_args['BasicFlatfieldCorrectionPlugin']
            apply_flatfield_args = plugin_args['ApplyFlatfield']
            apply_flatfield_args['outDir'] = basic_flatfield_args['inpDir']

        # Run the montage plugin for specified subset
        print('Running Montage Plugin on subset {}'.format(subset))
        montage_args = plugin_args['Montage']
        montage_args['inpDir'] = apply_flatfield_args['outDir']
        montage_args['filePattern'] = montage_args['filePattern'][subset]
        montage_args['outDir'] = montage_args['outDir'][subset]
        # pprint(montage_args)
        status, container_hash = run_container(montage_args, volume)
        hashes.append(container_hash)
        if status != 'Succeeded':
            raise Exception('Status of container {} is {}'.format(
                container_hash, status)
                           )
        # Run the assembler plugin for specified subset
        print('Running Image Assembler Plugin on subset {}'.format(subset))
        assembler_args = plugin_args['ImageAssembler']
        assembler_args['imgPath'] = apply_flatfield_args['outDir']
        assembler_args['stitchPath'] = montage_args['outDir']
        assembler_args['outDir'] = assembler_args['outDir'][subset]
        # pprint(assembler_args)
        status, container_hash = run_container(assembler_args, volume)
        status, hashes.append(container_hash)
        if status != 'Succeeded':
            raise Exception('Status of container {} is {}'.format(
                container_hash, status)
                           )

        # Run the precompute plugin for specified subset
        print('Running Precompute Slide Plugin on subset {}'.format(subset))
        precompute_args = plugin_args['PolusPrecomputeSlidePlugin']
        precompute_args['inpDir'] = assembler_args['outDir']
        precompute_args['filePattern'] = precompute_args['filePattern'][subset]
        precompute_args['outDir'] = precompute_args['outDir'][subset]
        # pprint(precompute_args)
        status, container_hash = run_container(precompute_args, volume)
        hashes.append(container_hash)
        if status != 'Succeeded':
            raise Exception('Status of container {} is {}'.format(
                container_hash, status)
                           )

        return True, subset, hashes

    except Exception:
        print('Subset {} failed'.format(subset))
        return False, subset, hashes


def common_parent(paths):
    """
    Returns the deepest common parent of a list of paths.
    """
    parents = [
        [parent for parent in Path(path).parents][:-1]
        for path in paths]

    for parent_list in parents:
        parent_list.reverse()

    common = None

    for level in zip(*parents):
        if len(set(level)) == 1:
            common = level[0]
        else:
            break

    return common


def stop_containers(hashes):
    """
    Stops all specified containers.
    """
    stopped_containers = []
    for h in hashes:
        cmd = 'docker stop {}'.format(h)
        stopped_containers.append(
            subprocess.check_output(
                cmd, shell=True
            ).decode(sys.stdout.encoding).strip()
        )
        # print('Stopped container {}'.format(h))

    return stopped_containers


def parse_log(log_path):

    re_subset = re.compile(r'(?<=subset\s)[0-9]+')
    success = {}

    with open(log_path) as fr:
        for line in fr.readlines():

            subset = int(re_subset.search(line).group(0))

            if 'ERROR' in line:
                success[subset] = False

            else:
                success[subset] = True

    complete_subsets = [subset for subset, status in success.items() if status]
    failed_subsets = [subset for subset, status in success.items() if not status]

    return complete_subsets, failed_subsets


def parse_output(outDir):

    complete = []
    incomplete = []
    precompute_out = outDir.joinpath('PolusPrecomputeSlidePlugin')

    for subset in precompute_out.iterdir():

        s = int(''.join([i for i in str(subset.name) if i.isnumeric()]))

        if list(subset.iterdir()):
            complete.append(s)

        else:
            incomplete.append(s)

    complete.sort()
    incomplete.sort()

    return complete, incomplete


class Visualization_UI:
    """
    A class to display a UI of the visualization pipeline.
    """

    def __init__(self):

        # Define dashboard style and stdout
        self.layout = widgets.Layout(width='auto', height='40px')
        self.style = {'description_width': 'initial'}
        self.out = widgets.Output(layout={'border': '1px solid black'})

        # Define variable to track dashboard workflow
        self.running = False
        self.valid_inputs = False

        self.hashes = []

        # Define the pipeline plugin workflow
        self.workflow = {
            'BasicFlatfieldCorrectionPlugin': None,
            'ApplyFlatfield': None,
            'Montage': None,
            'ImageAssembler': None,
            'PolusPrecomputeSlidePlugin': None,
            'smp-training': None
        }

        # Define the user arguments required for the pipeline
        self.user_args = {
            'inpDir': '/home/jovyan/shared/data/eastman_subset/input/',
            'outDir': '/home/jovyan/shared/data/file-server/pyramids/visualization-pipeline/test-v023/',
            'logPath': '/home/jovyan/shared/notebooks/pipelines/visualization/logs/test-v023.log',
            'filePattern': 'p{ppp}_x{xx}_y{yy}_wx{t}_wy{z}_c{c}.ome.tif',
            'wellVar': 'tz',
            'subsetBy': 'p',
            'depthVar': 'c',
            'layout': 'tz,xy',
            'flipAxis': 'z',
            'subsets': 'all',
        }

        # Define the pipeline's user input widgets
        self.arg_widgets = [
            widgets.Text(
                value=self.user_args[f],
                description=f,
                layout=self.layout,
                style=self.style
            )
            for f in self.user_args.keys()
        ]

        # Define ability for user to run flatfield or not
        self.run_flatfield = widgets.Checkbox(
            value=True,
            description='Run Flatfield Correction',
            disabled=False
        )

        self.pyramid_type = widgets.RadioButtons(
            options=['image', 'segmentation'],
            value='image',
            description='Image Type:',
            disabled=False
        )

        self.center_fov = widgets.RadioButtons(
            options=['normal', 'center fov'],
            value='normal',
            description='Flatfield Correction:',
            disabled=False,
            layout=self.layout,
            style={'description_width': 'initial'}
        )

        # Define the save inputs button
        self.save_args = widgets.Button(
            value=False,
            description='Save Inputs',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Save the Inputs',
            icon='save'  # (FontAwesome names without the `fa-` prefix)
        )

        self.run_pipeline = widgets.Button(
            value=True,
            description='Run Pipeline',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Runs the pipeline with current inputs (make sure to save inputs first)',
            icon='play'  # (FontAwesome names without the `fa-` prefix)
        )

        self.resume_pipeline = widgets.Button(
            value=True,
            description='Resume Pipeline',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Resumes the pipeline with current inputs (make sure to save inputs first)',
            icon='step-forward'  # (FontAwesome names without the `fa-` prefix)
        )

        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=None,
            description='Subsets Processed:',
            style=dict(description_width='initial'),
            layout=self.layout
            )

        # Define the method to run on button clicks
        self.save_args.on_click(self.on_save_clicked)
        self.run_pipeline.on_click(self.on_run_clicked)
        self.resume_pipeline.on_click(self.on_resumed_clicked)

        self.w = [
            *self.arg_widgets,
            self.run_flatfield,
            self.center_fov,
            self.pyramid_type,
            self.save_args,
            self.run_pipeline,
            self.resume_pipeline,
            self.progress_bar,
            self.out
        ]

    def display_widgets(self):
        """
        A method to display the widgets
        """

        display(
            *self.w
        )

    def on_save_clicked(self, b):
        """
        A method to save the users inputs.
        """
        with self.out:
            print('Saving args...')
            for w in self.arg_widgets:
                arg = w.description
                value = w.value
                self.user_args[arg] = value
                print('{} = {}'.format(arg, value))

            if self.run_flatfield.value:
                if self.center_fov.value == 'normal':
                    print('You have selected to run a normal Flatfield Correction')
                else:
                    print('You have selected to run a center FOV Flatfield Correction')

            else:
                print('You have selected to NOT run Flatfield Correction')

            if self.pyramid_type.value == 'image':
                print('You have indicated you have a non-segmented image')

            else:
                print('You have indicated you have a segmented image')

            print('Validating inputs...')
            self.valid_inputs = self.validate_inputs()

    def validate_inputs(self):
        """
        A method to validate the user's inputs.
        """
        with self.out:
            inpDir = Path(self.user_args['inpDir'])
            outDir = Path(self.user_args['outDir'])

            # Validate inpDir
            if not inpDir.exists():
                print('\n***WARNING***\nThe inpDir {} does not exist'.format(inpDir))
                return False  

            # TODO: Make sure inpDir and outDir either in:
            # /shared/data/ or /shared/notebooks/ or /work/

            if self.user_args['subsetBy'] in self.user_args['layout']:
                print('The subsetBy variable cannot be in the layout input')
                return False

            if self.user_args['depthVar'] in self.user_args['layout']:
                print('The depthVar variable cannot be in the layout input')
                return False

            # Transform layout variable to correct format
            layout = self.user_args['layout']
            layout = [g.strip() for g in layout.split(',')]

            # Check each layout grouping length
            for g in layout:
                if len(g) > 2:
                    print('Each layout group must be at most 2 variables')
                    return False
            self.user_args['layout'] = ','.join(layout)

            print('Inputs validated!')
            return True

    def on_run_clicked(self, b):
        """
        A method to start the pipeline on click
        """ 

        if self.running:
            with self.out:
                print('Pipeline already running, you must kill the pipeline to restart')

        elif not self.valid_inputs:
            with self.out:
                print('The inputs must be validated before running the pipeline, click Save Inputs to validate')

        else:
            self.running = True

            with self.out:
                print('Starting the pipeline...')
                print('Inferring plugin inputs and creating output file locations')
                self.generate_plugin_inputs()
                print('Running containers...')

            self.subsets = [i for i in range(len(self.filePatterns))]
            map_args = [self.plugin_args.copy() for subset in self.subsets]

            for subset in self.subsets:
                map_args[subset]['subset'] = subset

            if self.user_args['subsets'] != 'all':
                to_process = [int(s) for s in self.user_args['subsets'].split(',')]
                map_args = [map_args[s] for s in to_process]

            self.progress_bar.max = len(map_args)
            log_path = self.user_args['logPath']

            self.hashes.extend(parallelize_workflow(map_args, self, log_path))

            self.running = False

            if 'file-server' in self.user_args['outDir']:
                with self.out:
                    links = self.generate_aviator()
                    print('AVIATOR Links:')
                    for l in links:
                        print(l)

    def on_resumed_clicked(self, b):
        """
        A method to resume the pipeline
        """

        out_dir = Path(self.user_args['outDir'])

        with self.out:
            assert out_dir.exists(), 'The output directory for resuming a pipeline must already exist'

        if self.running:
            with self.out:
                print('Pipeline already running, you must kill the pipeline to restart')

        elif not self.valid_inputs:
            with self.out:
                print('The inputs must be validated before running the pipeline, click Save Inputs to validate')

        else:
            self.running = True

            with self.out:
                log_path = self.user_args['logPath']
                print('Parsing {}'.format(out_dir))

                # complete_subsets, failed_subsets = parse_log(log_path)
                complete_subsets, failed_subsets = parse_output(out_dir)

                print('Inferring plugin inputs and output file locations...')
                self.generate_plugin_inputs(resume=True)

                map_args = []
                self.subsets = []

                if self.user_args['subsets'] != 'all':
                    user_subsets = [int(s) for s in self.user_args['subsets'].split(',')]

                for i in range(len(self.filePatterns)):
                    if i not in complete_subsets:
                        if self.user_args['subsets'] == 'all' or i in user_subsets:
                            plugin_args = self.plugin_args.copy()
                            plugin_args['subset'] = i
                            map_args.append(plugin_args)
                            self.subsets .append(i)

                self.progress_bar.max = len(map_args)
                log_path = self.user_args['logPath']

                print('Resuming subsets : {}'.format(self.subsets))
                print('Starting containers...')
                self.hashes.extend(parallelize_workflow(map_args, self, log_path))

                self.running = False

                if 'file-server' in self.user_args['outDir']:
                    with self.out:
                        links = self.generate_aviator()
                        print('AVIATOR Links:')
                        for l in links:
                            print(l)

    def generate_outdir(self, resume):

        self.outDir = Path(self.user_args['outDir'])

        if not resume:
            os.mkdir(self.outDir)

        flatfield_plugins = [
            'BasicFlatfieldCorrectionPlugin', 'ApplyFlatfield'
        ]

        # Make plugin out directories
        for plugin in self.workflow:

            if plugin in flatfield_plugins and not self.run_flatfield.value:
                continue

            else:
                sub_dir = self.outDir.joinpath(plugin)

            if not resume:
                os.mkdir(sub_dir)

            self.workflow[plugin] = []

            # For each data subset create a subset directory
            for i in range(len(self.filePatterns)):

                subset_dir = sub_dir.joinpath('subset{}'.format(str(i)))
                self.workflow[plugin].append(subset_dir)

                if resume:
                    continue

                else:
                    os.mkdir(subset_dir)

    # TODO: Assemble patterns should be gerneated from actual file list
    def generate_assemble_pattern(self, groupBy):
        """
        Generates assembled patterns, similair to a vector pattern
        """
        self.assemblePatterns = self.filePatterns.copy()
        self.assembleFlatfieldPatterns = self.flatfieldPatterns.copy()

        # Iterate over the groupBy variables
        for v in groupBy:
            # Define the re pattern
            re_pattern = re.compile(r'\{(' + v + r'+)\}')
            # Define the min and max unique values of the variable
            v_unique = self.fp.uniques[v]
            v_min = min(v_unique)
            v_max = max(v_unique)
            num_dig = len(str(v_max))
            # Define the pattern to sub in for the variable
            v_pattern = '(' + str(v_min).zfill(num_dig) + '-' + str(v_max) + ')'

            # Iterate over all file patterns (one for each data subset)
            for i, pattern in enumerate(self.assemblePatterns):
                assemblePattern = re_pattern.sub(v_pattern, pattern)
                self.assemblePatterns[i] = assemblePattern

            # Iterate over all file patterns (one for each data subset)
            for i, pattern in enumerate(self.assembleFlatfieldPatterns):
                assemblePattern = re_pattern.sub(v_pattern, pattern)
                self.assembleFlatfieldPatterns[i] = assemblePattern

        self.assemblePatterns = ['"' + p + '"' for p in self.assemblePatterns]
        self.assembleFlatfieldPatterns = ['"' + p + '"' for p in self.assembleFlatfieldPatterns]

    def generate_plugin_inputs(self, resume=False):

        self.fp = FilePattern(
            self.user_args['inpDir'], self.user_args['filePattern']
        )

        flatfield_pattern = self.user_args['filePattern']

        if self.center_fov.value != 'normal':
            # Find range of well variables
            for v in self.user_args['wellVar']:
                # Define the regular expression to parse the well variables
                well_re = re.compile(r'\{(' + v + r'+)\}')
                uniques = self.fp.uniques[v]
                idx = len(uniques)//2
                flatfield_pattern = well_re.sub(str(uniques[idx]), flatfield_pattern)

        if self.user_args['subsetBy']:

            # Define the variable which the data will be split
            v = self.user_args['subsetBy']

            # Define the regular expression to parse the subset variable
            subset_re = re.compile(r'\{(' + v + r'+)\}')
            pattern = self.user_args['filePattern']

            # Find the subset variable and its length
            subset_var = subset_re.search(pattern).group(1)
            subset_len = len(subset_var)

            # Get the subset images
            # subsets = self.subset_images()

            # Generate the filepatterns for each data subset
            self.filePatterns = [
                subset_re.sub(str(value).zfill(subset_len), pattern)
                for value in self.fp.uniques[v]
            ]

            self.flatfieldPatterns = [
                subset_re.sub(str(value).zfill(subset_len), flatfield_pattern)
                for value in self.fp.uniques[v]
            ]

            self.filePatterns.sort()
            self.flatfieldPatterns.sort()
            groupBy = self.fp.variables.replace(v, '')

        else:
            self.filePatterns = [self.fp]
            groupBy = self.fp.varaibles

        if self.user_args['depthVar']:
            groupBy = groupBy.replace(self.user_args['depthVar'], '')

        with self.out:
            print('Generating assembled file patterns')
        # Generate the assembled patterns for each subset
        self.generate_assemble_pattern(groupBy)

        with self.out:
            print('Generating output directories')
        self.generate_outdir(resume)

        # Define the contianer volumes
        # jvolume = str(common_parent([ui.outDir, ui.final_dir]))
        jvolume = '/' + '/'.join(common_parent([ui.outDir]).parts[1:5]) + '/'
        dvolume = jvolume.replace('/home/jovyan', '/opt')
        self.volume = jvolume + ':' + dvolume

        # Define the plugin inputs
        self.basic_flatfield_args = {
            'container': 'polusai/basic-flatfield-correction-plugin:1.3.1',
            'inpDir': self.user_args['inpDir'],
            'filePattern': self.flatfieldPatterns,
            'darkfield': True,
            'photobleach': True,
            'groupBy': groupBy,
            'outDir': self.workflow['BasicFlatfieldCorrectionPlugin']
        }

        brightPatterns = [pattern.replace('.ome.tif', '_flatfield.ome.tif')
                          for pattern in self.assembleFlatfieldPatterns]

        darkPatterns = [pattern.replace('.ome.tif', '_darkfield.ome.tif')
                        for pattern in self.assembleFlatfieldPatterns]

        self.apply_flatfield_args = {
            'container': 'polusai/apply-flatfield-plugin:1.2.0',
            'imgDir': self.user_args['inpDir'],
            'imgPattern': self.filePatterns,
            'brightPattern': brightPatterns,
            'darkPattern': darkPatterns,
            'outDir': self.workflow['ApplyFlatfield']
        }

        self.montage_args = {
            'container': 'polusai/montage-plugin:0.4.0',
            'filePattern': self.filePatterns,
            'layout': self.user_args['layout'],
            'imageSpacing': '1',
            'gridSpacing': '20',
            'flipAxis': self.user_args['flipAxis'],
            'outDir': self.workflow['Montage']
        }

        self.assembler_args = {
            'container': 'polusai/image-assembler-plugin:1.1.5',
            'timesliceNaming': True,
            'outDir': self.workflow['ImageAssembler']
        }

        self.precompute_args = {
            'container': 'polusai/precompute-slide-plugin:1.5.0',
            'pyramidType': 'Zarr',
            'filePattern': self.assemblePatterns,
            'outDir': self.workflow['PolusPrecomputeSlidePlugin']
        }

        if self.pyramid_type == 'image':
            self.precompute_args['imageType'] = 'image'

        else:
            self.precompute_args['imageType'] = 'segmentation'

        self.plugin_args = {
            'BasicFlatfieldCorrectionPlugin':  self.basic_flatfield_args,
            'ApplyFlatfield': self.apply_flatfield_args,
            'Montage': self.montage_args,
            'ImageAssembler': self.assembler_args,
            'PolusPrecomputeSlidePlugin': self.precompute_args,
            'volume': self.volume,
            'run_flatfield': self.run_flatfield.value
        }

    def generate_aviator(self):
        """
        Generates an AVIATOR link for each data subset.
        """
        links = []
        base = 'https://avivator.gehlenborglab.org/?image_url=https://files.scb-ncats.io/pyramids/'

        for subset in self.workflow['PolusPrecomputeSlidePlugin']:
            for img in subset.iterdir():
                parts = list(img.parts)
                pyramid_index = parts.index('pyramids')
                path = '/'.join(parts[pyramid_index+1:])
                link = base + '/' + path + '/'
                links.append(link)

        return links

    def _clean(self):
        """
        Permanently deletes all workflow output files.
        """
        shutil.rmtree(self.outDir)
        # shutil.rmtree(self.final_dir)
        with self.out:
            print('Removed {}'.format(self.outDir))
            # print('Removed {}'.format(self.final_dir))

    def _stop_containers(self):
        """
        Stops all the containers associated with the workflow.
        """
        for h in self.hashes:
            cmd = 'docker stop {}'.format(h)
            stop = subprocess.check_output(cmd, shell=True).decode(sys.stdout.encoding).strip()
            with self.out:
                print('Stopped container {}'.format(h))

    def start(self):
        """
        A method to start the interactive dashboard.
        """

        display(widgets.interactive(self.display_widgets))


# Define and start the dashboard
ui = Visualization_UI()
ui.start()

interactive(children=(Output(),), _dom_classes=('widget-interact',))